In [ ]:
# import necessary libraries
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM


# Load Article from wikipedia

In [ ]:
loader = WikipediaLoader(query="indian constitution",load_max_docs=30) # Load Wikipedia articles related to the Indian Constitution
docs = loader.load()

#  Split text into chunks

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30) # Split text into chunks
# Split the documents into smaller chunks for better processing
chunks = splitter.split_documents(docs)

# Load the  Embedding model

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") # Load the embedding model


<ipython-input-9-2734244670>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Create the vector embedding

In [ ]:
vectorstore = FAISS.from_documents(docs, embedding_model) # Create the vector embedding

In [ ]:
retriever = vectorstore.as_retriever() # Create a retriever from the vector store

# Load the model

In [ ]:
model_id = "microsoft/phi-2" # Load the model

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)



# Define the Query from the document

In [ ]:
  query ="indian constitution history?" # Define the query for which we want to retrieve relevant documents
print(f"Retrieving chunks for: {query}")

retriever = vectorstore.as_retriever()
relevant_docs = retriever.get_relevant_documents(query)

# Concatenate the content of retrieved documents
context = "\n".join([doc.page_content for doc in relevant_docs])

print("\nRetrieved Context:\n")
print(context)

Retrieving chunks for: indian constitution history?

Retrieved Context:

The Constitution of India is the supreme legal document of India, and the longest written national constitution in the world. The document lays down the framework that demarcates fundamental political code, structure, procedures, powers, and duties of government institutions and sets out fundamental rights, directive principles, and the duties of citizens. 
It espouses constitutional supremacy (not parliamentary supremacy found in the United Kingdom, since it was created by a constituent assembly rather than Parliament) and was adopted with a declaration in its preamble. Although the Indian Constitution does not contain a provision to limit the powers of the parliament to amend the constitution, the Supreme Court in Kesavananda Bharati v. State of Kerala held that there were certain features of the Indian constitution so integral to its functioning and existence that they could never be cut out of the constitution